In [546]:
import numpy as np
import networkx as nx
import meshplot as mp
from copy import deepcopy

In [547]:
class Node:
    """
    Class Node
    """
    def __init__(self, value, radius = None, left = None, right = None, position = None, cl_prob= None, ce = None, mse = None):
        self.left = left
        self.data = value
        self.radius = radius
        self.position = position
        self.right = right
        self.prob = cl_prob
        self.mse = mse
        self.ce = ce
        self.children = [self.left, self.right]
    
    def traverseInorder(self, root):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            self.traverseInorder(root.left)
            print (root.data, root.radius)
            self.traverseInorder(root.right)

    def height(self, root):
        # Check if the binary tree is empty
        if root is None:
            return 0 
        # Recursively call height of each node
        leftAns = self.height(root.left)
        rightAns = self.height(root.right)
    
        # Return max(leftHeight, rightHeight) at each iteration
        return max(leftAns, rightAns) + 1

    def serialize(self, root):
        def post_order(root):
            if root:
                post_order(root.left)
                post_order(root.right)
                ret[0] += str(root.data)+'_'+ str(root.radius) +';'
                
            else:
                ret[0] += '#;'           

        ret = ['']
        post_order(root)
        return ret[0][:-1]  # remove last ,
    def toGraph( self, graph, index, dec, proc=True):
        
        
        radius = np.array(self.radius)
        if dec:
            radius= radius[0]
        #print("posicion", self.data, radius)
        #print("right", self.right)
        
        #graph.add_nodes_from( [ (index, {'posicion': radius[0:3], 'radio': radius[3] } ) ])
        graph.add_nodes_from( [ (self.data, {'posicion': radius[0:3], 'radio': radius[3] } ) ])
        

        if self.right is not None:
            #leftIndex = self.right.toGraph( graph, index + 1, dec)#
            self.right.toGraph( graph, index + 1, dec)#
            
            #graph.add_edge( index, index + 1 )
            graph.add_edge( self.data, self.right.data )
            #if proc:
            #    nx.set_edge_attributes( graph, {(index, index+1) : {'procesada':False}})
        
        if self.left is not None:
            #retIndex = self.left.toGraph( graph, leftIndex, dec )#
            self.left.toGraph( graph, 0, dec )#

            #graph.add_edge( index, leftIndex)
            graph.add_edge( self.data, self.left.data)
            #if proc:
            #    nx.set_edge_attributes( graph, {(index, leftIndex) : {'procesada':False}})
            
        #else:
            #return leftIndex
        #    return

        else:
            #return index + 1
            return

In [548]:
def plotTree( root, dec ):
    graph = nx.Graph()
    root.toGraph( graph, 0, dec)
    edges=nx.get_edge_attributes(graph,'procesada')
    print("edges",graph.edges.data() )
    p = mp.plot( np.array([ graph.nodes[v]['posicion'] for v in graph.nodes]), shading={'point_size':0.1}, return_plot=True)

    for arista in graph.edges:
        p.add_lines( graph.nodes[arista[0]]['posicion'], graph.nodes[arista[1]]['posicion'])

    return 

In [549]:
def getCountAtLevel (node, curr, desired, n):
    # If this node doesn't exist, must be zero.
    if node is None: 
        return 0

    # If this node is at desired level, must be one.
    if curr == desired: 
        n.append(node)
        return n

    # Otherwise sum of nodes at that level in left and right sub-trees.
    getCountAtLevel (node.left,  curr+1, desired, n)
    getCountAtLevel (node.right, curr+1, desired, n)
    return n

def numerar_nodos(root, count):
    if root is not None:
        root.data = len(count)
        count.append(1)
        numerar_nodos(root.right, count)
        numerar_nodos(root.left, count)
        #
        
        return 

def cr(root):
    h = root.height(root)
    for i in range(0, h):
        n = []
        #print("nodos en nivel ", i, getCountAtLevel(root, 0, i, n))
        c = getCountAtLevel(root, 0, i, n)
        #print("level", i)
        for j in range(len(c)):
            #c[j].radius = [i*3.+j+i,i*3.-j,i*1.,i*1.]
            #c[j].radius = [i*3.+j,i*1.+j,i*1.+j,i*1.]
            x = i*4.+1.*j
            y = i*1.+3.*j
            z = i*2.+5.*j
            r = i*1.
            #print("i,j", i, j)
            #print("radius", x,y,z,r)
            c[j].radius = [x,y,z,r]

In [550]:
def generarArbol( profundidad ):
    
    if profundidad == 0:
        return [ None ]
    elif profundidad == 1:
        return [ Node( 4 ) ]
    else:
        subarboles = [ generarArbol(profundidad= n) for n in range(0, profundidad )]
        res = [ ]
        for grande in subarboles[-1]:
            for s in subarboles[:-1]:
                for chico in s:
                    res.append( Node( 1, left = deepcopy(grande), right = deepcopy(chico) ) )
                    res.append( Node( 2, left = deepcopy(chico), right = deepcopy(grande) ))
        
        for s_i in subarboles[-1]:
            for s_d in subarboles[-1]:
                res.append( Node(3, left = deepcopy(s_i), right = deepcopy(s_d)))

        return res




In [551]:
g = generarArbol( 3 )
print(len(g) )
print( [ n.height(n) for n in g if n is not None])

21
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [552]:
ar = g[18]
ar.traverseInorder(ar)
count = []
numerar_nodos(ar, count)
print("///")
ar.traverseInorder(ar)
print("///")
print(ar.left.right.data)

4 None
3 None
4 None
3 None
4 None
1 None
///
5 None
3 None
4 None
0 None
2 None
1 None
///
4


In [553]:
for i in range (len(g)):
    arbol = g[i]
    count = []
    #print("antes de numerar")
    #arbol.traverseInorder(arbol)
    numerar_nodos(arbol, count)
    #print("despues de numerar")
    #arbol.traverseInorder(arbol)
    cr (arbol)
    #print("con radio")
    #arbol.traverseInorder(arbol)
    serial = arbol.serialize(arbol)
    #print("serialized", serial)
    #write serialized string to file
    file = open("./Trees/prof3/tree" + str(i) +".dat", "w")
    file.write(serial)
    file.close() 


In [554]:
def generarArbolRandom( profundidad, proba ):
    
    raiz = Node(1, left = None, right = None)
    cola = [ (raiz, profundidad) ]

    while len(cola) != 0:
        nodoActual, n = cola.pop(0)

        if n == 1:
            continue
        else:
            hijo1 = Node(1, left = None, right = None) if np.random.rand() > proba else None
            hijo2 = Node(1, left = None, right = None) if np.random.rand() > proba else None

            if hijo1 is None and hijo2 is None:
                if np.random.rand() > 0.5:
                    hijo1 = Node(1, left = None, right = None)
                else:
                    hijo2 = Node(1, left = None, right = None)

            nodoActual.left = hijo1
            nodoActual.right = hijo2

            [ cola.append( (hijo, n - 1)) for hijo in [hijo1, hijo2] if hijo is not None ]

    return raiz

In [555]:
root = generarArbolRandom(4, 0.5)
count = []
numerar_nodos(root, count)
cr (root)

In [556]:
plotTree(root, False)

edges [(0, 1, {}), (1, 2, {}), (2, 3, {}), (2, 4, {})]


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(6.5, 3.0,…

In [557]:
for j in range (200):
    root = generarArbolRandom(24, 0.8)
    count = []
    numerar_nodos(root, count)
    cr (root)
    serial = root.serialize(root)
    #write serialized string to file
    file = open("./Trees/prof24/tree" + str(j) +".dat", "w")
    file.write(serial)
    file.close() 


In [558]:
plotTree(g[4], False)

edges [(0, 1, {}), (1, 2, {})]


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(4.0, 1.0,…

In [559]:
g[8].traverseInorder(g[8])

3 [8.0, 2.0, 4.0, 2.0]
1 [4.0, 1.0, 2.0, 1.0]
2 [9.0, 5.0, 9.0, 2.0]
0 [0.0, 0.0, 0.0, 0.0]


In [560]:
g[9].traverseInorder(g[9])

0 [0.0, 0.0, 0.0, 0.0]
3 [8.0, 2.0, 4.0, 2.0]
1 [4.0, 1.0, 2.0, 1.0]
2 [9.0, 5.0, 9.0, 2.0]


In [561]:
g[10].right.data


1

In [562]:
g[10].right.left
g[10].traverseInorder(g[10])

4 [8.0, 2.0, 4.0, 2.0]
2 [4.0, 1.0, 2.0, 1.0]
3 [9.0, 5.0, 9.0, 2.0]
0 [0.0, 0.0, 0.0, 0.0]
1 [5.0, 4.0, 7.0, 1.0]
